In [1]:
import geopandas as gpd
import pandas as pd
import pandas
import math
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon
from geographiclib.geodesic import Geodesic
from shapely.ops import split
from shapely import ops
import shapely
from pyproj import Proj, Transformer
import warnings;
warnings.filterwarnings('ignore');

In [2]:
buildings= gpd.read_file('../tests/data/buildings.geojson')
road= gpd.read_file('../tests/data/road_feb.geojson')

In [3]:
# for d in buildings.columns:
#     print(d)
road['end'] = None
road['start'] = None
road['start_point_x'] = None
road['start_point_y'] = None
road['end_point_x'] = None
road['end_point_y'] = None
road['length'] = None
road['bearing'] = None
road['intersected'] = None
road['mother_road']=None

buildings['road_direction'] = 'None'
buildings['from_mother_road_direction'] = 'None'

buildings['distance2_intersecting_road'] = 0
buildings['road_length_d_code'] = ''

# print(buildings)
# print(road)

In [4]:
class bdcPoint:
    	
	def __init__(self):
		
		self.x = 0
		self.y = 0

# Constant integers for directions
RIGHT = 1
LEFT = -1
ZERO = 0

def directionOfPoint(A, B, P):
    """[Provides direction of the point in respect to line]

    Args:
        A ([type]): [Starting Point of Line]
        B ([type]): [Ending Point of Line]
        P ([type]): [Point of Interest]

    Returns:
        [Direction(int)]: [Left,Right and Zero : -1,1,0 ( Left direction, Right Direction , On Line)]
    """
    global RIGHT, LEFT, ZERO
	
	# Subtracting co-ordinates of
	# point A from B and P, to
	# make A as origin
 
    B.x -= A.x
    B.y -= A.y
    P.x -= A.x
    P.y -= A.y

    # Determining cross Product
    """"The Cross-Product has an interesting Property which will be used to determine direction of a point from a line segment. That is, the cross-product of two points is positive if and only if the angle of those point at origin (0, 0) is in counter-clockwise. And conversely the cross-product is negative if and only if the angle of those point at origin is in clockwise direction."""
    cross_product = B.x * P.y - B.y * P.x
    # Return RIGHT if cross product is positive
    if (cross_product > 0):
        return RIGHT
        
    # Return LEFT if cross product is negative
    if (cross_product < 0):
        return LEFT

    # Return ZERO if cross product is zero
    return ZERO

def testLeftRightofLine(start,end,poi):
    d=(poi.x-start.x)*(end.y-start.y)-(poi.y-start.y)*(end.x-start.x)
    if d>0:
        #left
        return -1
    if d<0:
        #right
        return 1
    else:
        return 0

def findMotherRoad2AssociatedRoadDirection(mother_road,associated_road):
    left_buffered_line=mother_road.buffer(0.0001,single_sided=True)
#     right_buffered_line=direction_updated_roi.buffer(0.0007,single_sided=True)
    if left_buffered_line.intersects(associated_road.buffer(0.00001)) == True:
        direction=-1
#         print("left")
    else:
        direction=1
#         print("right")
    return direction

def addDirection_BuildingCode(direction,building_length,road_length,index,road_index):
    #right-even, left-odd number 
    # print(length)
    building_length=int(building_length)
    if road_length < 60:
        associated_road=road.iloc[road_index]

        if associated_road.mother_road == 'start':
            mother_road_code=associated_road.start
        else :
            mother_road_code=associated_road.end

        mother_road= road.loc[road['NEW'] == mother_road_code]

        mr_geom= mother_road.geometry
        d = {'col1': ['name1'], 'geometry': [Point(associated_road.start_point_x, associated_road.start_point_y)]}
        motherroad_start_point = {'col1': ['name1'], 'geometry': [Point(mother_road.start_point_x, mother_road.start_point_y)]}
        
        gdf = gpd.GeoDataFrame(d, crs=4326)
        mother_df = gpd.GeoDataFrame(motherroad_start_point, crs=4326)
        
        sp_point=gdf.iloc[0]
        mother_st_p=mother_df.iloc[0]
        for line in mr_geom:
            # print(line)
            for l in line:
                mother_line=l
                break

        result=split(mother_line,sp_point.geometry)
        for r  in result:
            splitted_line=r
            splitted_line.srid = 4326
            if splitted_line.contains(mother_st_p.geometry):
                break
            else:
                continue
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)
#         mother_road_direction=findMotherRoad2AssociatedRoadDirection(mother_road.geometry,associated_road.geometry)
        buildings.at[index,'road_length_d_code']=str(int(geom_transformed.length))+"/"
        
#         if (mother_road_direction == 1):
#             # print("Right Direction")
#             buildings.at[index,'from_mother_road_direction']="right"
#             buildings.at[index,'road_length_d_code']=str(int(round_up_to_even(geom_transformed.length)))+"/"
#         if (mother_road_direction == -1):
#             # print("left Direction")
#             buildings.at[index,'from_mother_road_direction']="left"
#             buildings.at[index,'road_length_d_code']=str(int(round_up_to_odd(geom_transformed.length)))+"/"
            
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_even(building_length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_odd(building_length)))
        else:
            print("On  Line")
            buildings.at[index,'road_direction']="on_line"
    else :
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_even(building_length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_odd(building_length)))
        else:
            print("On  Line")
            buildings.at[index,'road_direction']="on_line"
    
    
#     print(buildings.at[index,'road_length_d_code'])
            
def getStartEndPoint(line,type):
    if len(line)==1 or type=="road" :
       
        df_line = line.geometry.boundary
        # print(len(df_line))
        # print(line_bound)
        # df_line=line_bound.explode(index_parts=True)
        start_point,end_point=df_line[0],df_line[1]

        return start_point,end_point
    else:
        raise ValueError ("Line contains more than one code")

def getLineLength(line):
    projected_line = line.to_crs(epsg=32644)
    length= projected_line.geometry.length
    float_length=length.tolist()
    return float_length

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

def round_up_to_odd(f):
    return math.ceil(f) // 2 * 2 + 1

def addStartEndPoint(start_point,end_point,index):
    
    road.at[index,'start_point_x']=float(start_point.x)
    road.at[index,'start_point_y']=float(start_point.y)
    
    road.at[index,'end_point_x']=float(end_point.x)
    road.at[index,'end_point_y']=float(end_point.y)

def ReviseStartEndDateWithBearing(bearing,index):
    # default direction is west to east , south to north 
    # print("ma revise garna aako dubai vettera")
    if bearing is None :
        bearing=0
    bearing=int(bearing)
    def assign_start():
#         print("start")
        
        road.at[index,'mother_road']='start'

    def assign_end():
    # east_west/west_east direction , assign to east
#         print("end")
        road.at[index,'mother_road']='end'

    
    if ((bearing >= 0) and (bearing<=45)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
        
    if ((bearing > 45) and (bearing<135)) :
        
        # east_west/west_east direction , assign to east
        assign_end()
    
    if ((bearing >= 135) and (bearing<=225)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
    if ((bearing > 225) and (bearing<315)) :
            
        # east_west/west_east direction , assign to east
        assign_end()
    if ((bearing >= 315) and (bearing<=360)) :
    #    North_South/south_north Direction, assign to south
        
        assign_start()      

def FindIntersectedFeatures(start_point,end_point,line_row,index_road):
    for index, row in road.iterrows():
        line=row.geometry        
        if str(row.NEW) != str(line_row.NEW):
            if line.distance(start_point) <  0.000001 :
                # print("start found "+ str(row.NEW))
                # print("mero start vetiyo guys ")
                road.at[index_road,'start']=str(row.NEW)
                if row.TYPE_CODE == int(line_row.TYPE_CODE-1):
                    road.at[index_road,'mother_road']='start'
    
            if line.distance(end_point) < 0.000001:
                # print("end found "+ str(row.NEW))
                
                road.at[index_road,'end']=str(row.NEW)
                    # print("mero end vetiyo guys ")
                if row.TYPE_CODE == int(line_row.TYPE_CODE-1):
                    road.at[index_road,'mother_road']='end'


def scan_I_feature(line_row,index_road):
#         print(line_row)
    
        if ((line_row.start!= None) and (line_row.end== None)) :
            road.at[index_road,'mother_road']='start'
            
        if ((line_row.start== None) and (line_row.end != None)) :
            
            road.at[index_road,'mother_road']='end'
#             print("assigned end")
    
        if ((line_row.start== None) and (line_row.end == None)) :
        #both start and end exist which means line is in between roads 
            road.at[index_road,'mother_road']='None'
            ReviseStartEndDateWithBearing(line_row.bearing,index_road)

                    
def CalculateBearing(start_point,end_point,index):
    brng = Geodesic.WGS84.Inverse(start_point.y, start_point.x, end_point.y, end_point.x)['azi1']
    road.at[index,'bearing']=brng

def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False    
    
def AssignIntersectedPoint(start_point_assumed,end_point_assumed,row,index):
#     print(row)
    if  row.mother_road :
#         print(row.mother_road)
        mother_road_direction = row.mother_road
        mother_road_code=None
        if mother_road_direction == 'start':
            mother_road_code=row.start
        elif mother_road_direction == 'end' :
#             print("end assign garna aako")
            mother_road_code=row.end
#         print(mother_road_code)
        mother_road= road.loc[road['NEW'] == mother_road_code].geometry
        
        
#         print(road.loc[road['NEW'] == mother_road_code].NEW)
        current_road = row.geometry
        mother_road_geom=mother_road.buffer(0.000001)
        for l in mother_road:
            for single in l:
                mother_road_geom=single
            break
#         print(mother_road_geom)
        for l in current_road:
            road_geom=l
            break
        # print(road_geom)
        # print(mother_road_geom)
        start_point=mother_road_geom.intersection(road_geom.buffer(0.000001))
#         print(start_point)
#         print(st)
#         geoseries=f"""<class 'geopandas.geoseries.GeoSeries'>"""
#         print(geoseries)
    
        if isinstance(start_point,pandas.core.series.Series) :
#             print(type(start_point))
#             print('not counted')
            
            road.at[index,'intersected']='No'
        else:
            road.at[index,'intersected']='Yes'
            
            try:
                x=start_point.x
                y=start_point.y
                road.at[index,'start_point_x']=float(x)
                road.at[index,'start_point_y']=float(y)              
            except:
                try:
                    for p in start_point:
                        x=p.x
                        y=p.y
                        break
                    road.at[index,'start_point_x']=x
                    road.at[index,'start_point_y']=y
                except:
                    try:
                        df_line = start_point.boundary
                        start=df_line[0]
                        road.at[index,'start_point_x']=start.x
                        road.at[index,'start_point_y']=start.y
                    except:
                        print('i didnt assigned')
                        pass
#                     road.at[index,'start_point_x']=0
#                     road.at[index,'start_point_y']=0
 

        
def get_building_projected_road_length(road_linked_wrt_point,row):
    road_code = row.NEW
#     print(road_linked_wrt_point)
    linked_road_geometry = road_linked_wrt_point.geometry
    building_geometry = row.geometry
    degree_length_for_building=linked_road_geometry.project(building_geometry)
    # 5m=0.00005 in degree 
    building_point_projected_on_road=linked_road_geometry.interpolate(degree_length_for_building) #end_always 
    for r in building_point_projected_on_road:
        building_point_projected_on_road_single_geom=r
        break
    try:
        projected_building_on_road = {'col1': ['name1'], 'geometry': [Point(building_point_projected_on_road_single_geom.x, building_point_projected_on_road_single_geom.y)]}
    except:
        print(projected_building_on_road)
        raise("found error")

    road_link_wrt_point_x=road_linked_wrt_point.start_point_x
    road_link_wrt_point_y=road_linked_wrt_point.start_point_y
    try:
        road_start_point = {'col1': ['name1'], 'geometry': [Point(road_link_wrt_point_x,road_link_wrt_point_y)]}

        projected_building_on_road_gdf = gpd.GeoDataFrame(projected_building_on_road, crs=4326)


        road_start_df = gpd.GeoDataFrame(road_start_point, crs=4326)


        splitting_point=projected_building_on_road_gdf.iloc[0]
        start_point=road_start_df.iloc[0]
#         print(start_point.geometry)
        for line in linked_road_geometry:
            # print(line)
            for l in line:
                mother_line=l
                break
        result=split(mother_line,splitting_point.geometry.buffer(0.0000001))

        for r  in result:
                splitted_line=r
                splitted_line.srid = 4326
                if splitted_line.buffer(0.00001).contains(start_point.geometry.buffer(0.0000001)) == True:

                    break
                else:
                    continue


        
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)

        clipped_length=geom_transformed.length
        return clipped_length ,splitted_line ,building_point_projected_on_road_single_geom,start_point.geometry,degree_length_for_building
    except:
        pass



    
                   


In [5]:
def CalculateTouchLine(index,row):
    
    # print(row)
    start_point,end_point=getStartEndPoint(row,"road")
    s=addStartEndPoint(start_point,end_point,index)
    y=CalculateBearing(start_point,end_point,index)
    z=FindIntersectedFeatures(start_point,end_point,row,index)
    n=scan_I_feature(row,index)
#     print(row)
    a=AssignIntersectedPoint(start_point,end_point,row,index)
    # break


In [21]:
i=1992
row=road.iloc[i]
CalculateTouchLine(i,row)
print(road.iloc[i])

i didnt assigned
OBJECTID                                                      2008
OBJECTID_1                                                    2008
Class                                                            D
Code                                                         D2776
Priority                                                       0.0
ROW                                                           20.0
Road_Nam_1                                                    None
Surface                                                    Earthen
Ward_No                                                       17.0
Width                                                         16.0
Shape_Leng                                                 0.00467
NEW                                                         TS3058
TYPE_CODE                                                        6
Shape_Le_1                                                 0.00467
geometry         (LINESTRING (83.523105013000

In [14]:
max_j=len(road.index)
i=0
print(max_j)
# for i in range(0,100):
#     row=road.iloc[i]
#     try:
#         a=CalculateTouchLine(i,row)
#         print((f"""i: {i}/ {max_j}"""))
#     except:
#         continue

while i < max_j :
    row=road.iloc[i]
    try:
        a=CalculateTouchLine(i,row)
        print((f"""i: {i}/ {max_j}"""))
        i=i+1
    except:
        print((f"""error: {i}/ {max_j}"""))
        i=i+1
        continue
    
    
# print(road.iloc[104])

2502
i: 0/ 2502
i: 1/ 2502
i: 2/ 2502
i: 3/ 2502
i: 4/ 2502
i: 5/ 2502
i: 6/ 2502
i: 7/ 2502
i: 8/ 2502
i: 9/ 2502
i: 10/ 2502
i: 11/ 2502
i: 12/ 2502
i: 13/ 2502
i: 14/ 2502
i: 15/ 2502
i: 16/ 2502
i: 17/ 2502
i: 18/ 2502
i: 19/ 2502
i: 20/ 2502
i: 21/ 2502
i: 22/ 2502
i: 23/ 2502
i: 24/ 2502
i: 25/ 2502
i: 26/ 2502
i: 27/ 2502
i: 28/ 2502
i: 29/ 2502
i: 30/ 2502
i: 31/ 2502
i: 32/ 2502
i: 33/ 2502
i: 34/ 2502
i: 35/ 2502
i: 36/ 2502
i: 37/ 2502
i: 38/ 2502
i: 39/ 2502
i: 40/ 2502
i: 41/ 2502
i: 42/ 2502
i: 43/ 2502
i: 44/ 2502
i: 45/ 2502
i: 46/ 2502
i: 47/ 2502
i: 48/ 2502
i: 49/ 2502
i: 50/ 2502
i: 51/ 2502
i: 52/ 2502
i: 53/ 2502
i: 54/ 2502
i: 55/ 2502
i: 56/ 2502
i: 57/ 2502
i: 58/ 2502
i: 59/ 2502
i: 60/ 2502
i: 61/ 2502
i: 62/ 2502
i: 63/ 2502
i: 64/ 2502
i: 65/ 2502
i: 66/ 2502
i: 67/ 2502
i: 68/ 2502
i: 69/ 2502
i: 70/ 2502
i: 71/ 2502
i: 72/ 2502
i: 73/ 2502
i: 74/ 2502
i: 75/ 2502
i: 76/ 2502
i: 77/ 2502
i: 78/ 2502
i: 79/ 2502
i: 80/ 2502
i: 81/ 2502
i: 82/ 2502
i: 83/ 25

i: 639/ 2502
i: 640/ 2502
i: 641/ 2502
i: 642/ 2502
i: 643/ 2502
i: 644/ 2502
i: 645/ 2502
i: 646/ 2502
i: 647/ 2502
i: 648/ 2502
i: 649/ 2502
i: 650/ 2502
i: 651/ 2502
i: 652/ 2502
i: 653/ 2502
i: 654/ 2502
i: 655/ 2502
i: 656/ 2502
i: 657/ 2502
i: 658/ 2502
i: 659/ 2502
i: 660/ 2502
i: 661/ 2502
i: 662/ 2502
i: 663/ 2502
i: 664/ 2502
i: 665/ 2502
i: 666/ 2502
i: 667/ 2502
i: 668/ 2502
i: 669/ 2502
i: 670/ 2502
i: 671/ 2502
i: 672/ 2502
i: 673/ 2502
i: 674/ 2502
i: 675/ 2502
i: 676/ 2502
i: 677/ 2502
i: 678/ 2502
i: 679/ 2502
i: 680/ 2502
i: 681/ 2502
i: 682/ 2502
i: 683/ 2502
i: 684/ 2502
i: 685/ 2502
i: 686/ 2502
i: 687/ 2502
i: 688/ 2502
i: 689/ 2502
i: 690/ 2502
i: 691/ 2502
i: 692/ 2502
i: 693/ 2502
i: 694/ 2502
i: 695/ 2502
i: 696/ 2502
i: 697/ 2502
i: 698/ 2502
i: 699/ 2502
i: 700/ 2502
i: 701/ 2502
i: 702/ 2502
i: 703/ 2502
i: 704/ 2502
i: 705/ 2502
i: 706/ 2502
i: 707/ 2502
i: 708/ 2502
i: 709/ 2502
i: 710/ 2502
i: 711/ 2502
i: 712/ 2502
i: 713/ 2502
i: 714/ 2502
i: 715/ 2502

i: 1250/ 2502
i: 1251/ 2502
i: 1252/ 2502
i: 1253/ 2502
i: 1254/ 2502
i: 1255/ 2502
i: 1256/ 2502
i: 1257/ 2502
i: 1258/ 2502
i: 1259/ 2502
i: 1260/ 2502
i: 1261/ 2502
i: 1262/ 2502
i: 1263/ 2502
i: 1264/ 2502
i: 1265/ 2502
i: 1266/ 2502
i: 1267/ 2502
i: 1268/ 2502
i: 1269/ 2502
i: 1270/ 2502
i: 1271/ 2502
i: 1272/ 2502
i: 1273/ 2502
i: 1274/ 2502
i: 1275/ 2502
i: 1276/ 2502
i: 1277/ 2502
i: 1278/ 2502
i: 1279/ 2502
i: 1280/ 2502
i: 1281/ 2502
i: 1282/ 2502
i: 1283/ 2502
i: 1284/ 2502
i: 1285/ 2502
i: 1286/ 2502
i: 1287/ 2502
i: 1288/ 2502
i: 1289/ 2502
i: 1290/ 2502
i: 1291/ 2502
i: 1292/ 2502
i: 1293/ 2502
i: 1294/ 2502
i: 1295/ 2502
i: 1296/ 2502
i: 1297/ 2502
i: 1298/ 2502
i: 1299/ 2502
i: 1300/ 2502
i: 1301/ 2502
i: 1302/ 2502
i: 1303/ 2502
i: 1304/ 2502
i: 1305/ 2502
i: 1306/ 2502
i: 1307/ 2502
i: 1308/ 2502
i: 1309/ 2502
i: 1310/ 2502
i: 1311/ 2502
i: 1312/ 2502
i: 1313/ 2502
i: 1314/ 2502
i: 1315/ 2502
i: 1316/ 2502
i: 1317/ 2502
i: 1318/ 2502
i: 1319/ 2502
i: 1320/ 2502
i: 132

i: 1836/ 2502
i: 1837/ 2502
i: 1838/ 2502
i: 1839/ 2502
i: 1840/ 2502
i: 1841/ 2502
i: 1842/ 2502
i: 1843/ 2502
i: 1844/ 2502
i: 1845/ 2502
i: 1846/ 2502
i: 1847/ 2502
i: 1848/ 2502
i: 1849/ 2502
i: 1850/ 2502
i: 1851/ 2502
i: 1852/ 2502
i: 1853/ 2502
i: 1854/ 2502
i: 1855/ 2502
i: 1856/ 2502
i: 1857/ 2502
i: 1858/ 2502
i: 1859/ 2502
i: 1860/ 2502
i: 1861/ 2502
i: 1862/ 2502
i: 1863/ 2502
i: 1864/ 2502
i: 1865/ 2502
i: 1866/ 2502
i: 1867/ 2502
i: 1868/ 2502
i: 1869/ 2502
i: 1870/ 2502
i: 1871/ 2502
i: 1872/ 2502
i: 1873/ 2502
i: 1874/ 2502
i: 1875/ 2502
i: 1876/ 2502
i: 1877/ 2502
i: 1878/ 2502
i: 1879/ 2502
i: 1880/ 2502
i: 1881/ 2502
i: 1882/ 2502
i: 1883/ 2502
i: 1884/ 2502
i: 1885/ 2502
i: 1886/ 2502
i: 1887/ 2502
i: 1888/ 2502
i: 1889/ 2502
i: 1890/ 2502
i: 1891/ 2502
i: 1892/ 2502
i: 1893/ 2502
i: 1894/ 2502
i: 1895/ 2502
i: 1896/ 2502
i: 1897/ 2502
i: 1898/ 2502
i: 1899/ 2502
i: 1900/ 2502
i: 1901/ 2502
i: 1902/ 2502
i: 1903/ 2502
i: 1904/ 2502
i: 1905/ 2502
i: 1906/ 2502
i: 190

i: 2410/ 2502
i: 2411/ 2502
i: 2412/ 2502
i: 2413/ 2502
i: 2414/ 2502
i: 2415/ 2502
i: 2416/ 2502
i: 2417/ 2502
i: 2418/ 2502
i: 2419/ 2502
i: 2420/ 2502
i: 2421/ 2502
i: 2422/ 2502
i: 2423/ 2502
i: 2424/ 2502
i: 2425/ 2502
i: 2426/ 2502
i: 2427/ 2502
i: 2428/ 2502
i: 2429/ 2502
i: 2430/ 2502
i: 2431/ 2502
i: 2432/ 2502
i: 2433/ 2502
i: 2434/ 2502
i: 2435/ 2502
i: 2436/ 2502
i: 2437/ 2502
i: 2438/ 2502
i: 2439/ 2502
i: 2440/ 2502
i: 2441/ 2502
i: 2442/ 2502
i: 2443/ 2502
i: 2444/ 2502
i: 2445/ 2502
i: 2446/ 2502
i: 2447/ 2502
i: 2448/ 2502
i: 2449/ 2502
i: 2450/ 2502
i: 2451/ 2502
i: 2452/ 2502
i: 2453/ 2502
i: 2454/ 2502
i: 2455/ 2502
i: 2456/ 2502
i: 2457/ 2502
i: 2458/ 2502
i: 2459/ 2502
i: 2460/ 2502
i: 2461/ 2502
i didnt assigned
i: 2462/ 2502
i: 2463/ 2502
i: 2464/ 2502
i: 2465/ 2502
i: 2466/ 2502
i: 2467/ 2502
i: 2468/ 2502
i: 2469/ 2502
i: 2470/ 2502
i: 2471/ 2502
i: 2472/ 2502
i: 2473/ 2502
i: 2474/ 2502
i: 2475/ 2502
i: 2476/ 2502
i: 2477/ 2502
i: 2478/ 2502
i: 2479/ 2502
i: 

In [17]:
print(road.iloc[2460])

OBJECTID                                                      2558
OBJECTID_1                                                    2558
Class                                                            D
Code                                                         D2753
Priority                                                       0.0
ROW                                                           20.0
Road_Nam_1                                                    None
Surface                                                    Earthen
Ward_No                                                       17.0
Width                                                         16.0
Shape_Leng                                                 0.00539
NEW                                                           None
TYPE_CODE                                                        0
Shape_Le_1                                                0.005393
geometry         (LINESTRING (83.49741575000007 27.59859204300

In [ ]:
def assign_building_code(index,row):  

    road_linked_wrt_point= road.loc[road['NEW'] == row.NEW]
#     print(road_linked_wrt_point)
    for l in road_linked_wrt_point.geometry:
        line_geom=l
        break

    my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
    geom_transformed = ops.transform(my_transformer.transform, line_geom)

    road_length=geom_transformed.length

    clipped_length,splitted_mother_road_side,building_point_projected_on_road_single_geom,start_point,degree_length_for_building=get_building_projected_road_length(road_linked_wrt_point,row)

    splitted_with_buffer_boi=split(splitted_mother_road_side,building_point_projected_on_road_single_geom.buffer(0.0001)) 

    for r  in splitted_with_buffer_boi:
        roi_line=r
        roi_line.srid = 4326
        if roi_line.buffer(0.00001).contains(start_point.buffer(0.0000001)) == False:
            break
        else:
            continue
#     print(roi_line)
#     print(shapely.geometry.mapping(roi_line))
    df_line = roi_line.boundary
    start_roi,end_roi=df_line[0],df_line[1]
#     print(start_roi)
#     print(end_roi)
#     print(building_point_projected_on_road_single_geom)
    if (round(start_roi.x,6) == round(building_point_projected_on_road_single_geom.x,4)) and (round(start_roi.y,4) == round(building_point_projected_on_road_single_geom.y,4)) :
        start_roi=end_roi
        end_roi=building_point_projected_on_road_single_geom
        print("start_end changed")

    road_index=road_linked_wrt_point.index[0]
#         road.at[road_index,'length']=length[0]       

    poi = row.geometry
    #constructing new geometry with new start and end for checking left and right 
    direction_updated_roi=LineString([start_roi,end_roi])

    left_buffered_line=direction_updated_roi.buffer(0.01,single_sided=True)
#     right_buffered_line=direction_updated_roi.buffer(0.0007,single_sided=True)
    if left_buffered_line.intersects(poi.buffer(0.00002)) == True:
        direction=-1
#         print("left")
    else:
        direction=1
#         print("right")
        
#     print(shapely.geometry.mapping(left_buffered_line))
#     print(shapely.geometry.mapping(poi.buffer(0.00004)))
    
    # break
#     print(clipped_length)
#     print(direction)
    buildings.at[index,'distance2_intersecting_road']=clipped_length
    road.at[road_index,'length']=road_length
    
    
    addDirection_BuildingCode(direction,clipped_length,road_length,index,road_index)
    
        
#     print("added building code")


In [ ]:
i=6507
row=buildings.iloc[i]
assign_building_code(i,row)
print(buildings.iloc[i])



In [ ]:
i=0
max_loop=len(buildings.index)
while i < max_loop:
    row=buildings.iloc[i]
    try:
        assign_building_code(i,row)
        print(buildings.iloc[i].road_length_d_code)
        print((f"""i: {i}/ {max_loop}"""))
        i=i+1
    except:
        print((f"""error: {i}/ {max_loop}"""))
        i=i+1
        continue
    

In [ ]:
buildings.to_file("output_building_1.geojson", driver="GeoJSON")

In [18]:
road.to_file("output_road_2.geojson", driver="GeoJSON")